In [1]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.vector_stores import ChromaVectorStore
from llama_index.readers.chroma import ChromaReader
from llama_index import StorageContext, load_index_from_storage, load_indices_from_storage
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index.llms import HuggingFaceLLM
from llama_index.node_parser import SentenceSplitter 
from llama_index.schema import MetadataMode
from IPython.display import Markdown, display
from llama_index import QueryBundle 
from peft import PeftModel, PeftConfig
from transformers import TextStreamer, GenerationConfig
from transformers import LlamaForCausalLM, LlamaTokenizer, LlamaTokenizerFast, BitsAndBytesConfig
from llama_index.response_synthesizers import (
    ResponseMode,
    get_response_synthesizer,
)
import chromadb
import transformers
import pandas as pd
import json 
import openai
import os
import getpass
import torch

In [2]:
default_path = os.getcwd()
data_path = os.path.join(default_path, '/rag/data')
model_path = os.path.join(default_path, '/rag/models')
config_path = os.path.join(default_path, '/rag/config')

In [3]:
model_dir = os.path.join(model_path, "mistral_origin")

In [4]:
tokenizer = AutoTokenizer.from_pretrained(os.path.join(model_dir, 'tokenizer'))   # LlamaTokenizer (x)  -> LlamaTokenizerFast (o)
model = AutoModelForCausalLM.from_pretrained(model_dir, torch_dtype=torch.float16, low_cpu_mem_usage=True) # , device_map="auto")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
g_device = torch.device("cuda") if torch.cuda.is_available() else "cpu"
model.to(g_device)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm()
  

In [6]:
generation_config = GenerationConfig(
    temperature=0.8,
    do_sample=True,
    top_p=0.95,
    max_new_tokens=512,
)

In [7]:
# col = ['qualification', 'desc', 'features']
col = ['card', 'loan', 'deposit']

In [8]:
vector_db = chromadb.HttpClient(host='192.168.0.146')

In [16]:
vector_db.list_collections()

[Collection(name=deposit), Collection(name=card), Collection(name=loan)]

In [17]:
card_collection = vector_db.get_or_create_collection("card")
loan_collection = vector_db.get_or_create_collection("loan")
deposit_collection = vector_db.get_or_create_collection("deposit")

In [20]:
deposit_collection.count()

1527

In [21]:
generation_config = GenerationConfig(
    temperature=0.8,
    do_sample=True,
    top_p=0.95,
    max_new_tokens=512,
)

In [11]:
model_name = 'kakaobank/kf-deberta-base'
embed_model = HuggingFaceEmbedding(model_name=model_name)

In [22]:
service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=None)

LLM is explicitly disabled. Using MockLLM.


In [23]:
parser = SentenceSplitter(chunk_size=512, chunk_overlap=30)   # SentenceSplitter(chunk_size=1024, chunk_overlap=20)

In [24]:
embedding_service = ServiceContext.from_defaults(node_parser=parser, embed_model=embed_model, llm=None)

LLM is explicitly disabled. Using MockLLM.


In [25]:
card_store = ChromaVectorStore(chroma_collection=card_collection)
loan_store = ChromaVectorStore(chroma_collection=loan_collection)
deposit_store = ChromaVectorStore(chroma_collection=deposit_collection)

In [26]:
# service_context 전달 안해주면 query 시 dimension 오류 발생 
card_idx = VectorStoreIndex.from_vector_store(card_store, service_context=embedding_service)
loan_idx = VectorStoreIndex.from_vector_store(loan_store, service_context=embedding_service)
deposit_idx = VectorStoreIndex.from_vector_store(deposit_store, service_context=embedding_service)

In [27]:
card_retriever = card_idx.as_retriever()
loan_retriever = loan_idx.as_retriever()
deposit_retriever = deposit_idx.as_retriever()

In [35]:
from llama_index.tools import RetrieverTool

card_tool = RetrieverTool.from_defaults(
    retriever=card_retriever, 
    description=(
        "카드 상품에 대해 물어볼 때 대답해줘"
    ),
)

In [39]:
loan_tool = RetrieverTool.from_defaults(
    retriever=loan_retriever,
    description=(
        "대출 상품에 대해 물어볼 때 대답해줘"
    ),
)

In [40]:
deposit_tool = RetrieverTool.from_defaults(
    retriever=deposit_retriever,
    description=(
        "예금 상품에 대해 질문할 때 대답해줘"
    ),
)

In [67]:
retriever = RouterRetriever(
    selector=EmbeddingSingleSelector.from_defaults(embed_model=embed_model),
    retriever_tools=[
        card_tool,
        loan_tool,
        deposit_tool,
    ],
    service_context=embedding_service
)

In [68]:
len(retriever.retrieve("청년들이 가입할 수 있는 대출 상품과 카드 상품 추천해줘"))

2

In [102]:
nodes = retriever.retrieve("예금 상품 추천해줘")

In [103]:
def get_info(retrieved_node):
    '''
    id, text, score 반환 
    '''
    id = retrieved_node.node.relationships['1'].node_id
    name = retrieved_node.node.relationships['1'].metadata['name']
    txt = retrieved_node.node.text 
    score = retrieved_node.score 
    return [id, name, txt, score] 

In [104]:
get_info(nodes[0]), get_info(nodes[1])

(['보증_1020',
  'm-정기적금',
  '월급의일부를모아목돈을만들어재테크의기반을만들고싶으신가요?그러나,많은투자상품가운데어떤상품을해야될지난감할때가있으시죠?망설이지마세요.재테크와목돈마련의시작은바로정기적금입니다.비과세종합저축으로도가능한웰컴저축은행의정기적금!재테크의시작!웰컴저축은행과시작하세요.예금자보호모바일뱅킹',
  0.5005266448258541],
 ['보증_1021',
  'e-정기적금',
  '월급의일부를모아목돈을만들어재테크의기반을만들고싶으신가요?그러나,많은투자상품가운데어떤상품을해야될지난감할때가있으시죠?망설이지마세요.재테크와목돈마련의시작은바로정기적금입니다.비과세종합저축으로도가능한웰컴저축은행의정기적금!재테크의시작!웰컴저축은행과시작하세요.예금자보호인터넷뱅킹',
  0.4967153542248623])

In [105]:
nodes[0].score > nodes[1].score

True

In [106]:
nodes

[NodeWithScore(node=TextNode(id_='6e4f10aa-0aaa-4645-831e-bdcc29866b8a', embedding=None, metadata={'category': 'deposit', 'name': 'm-정기적금', 'type': '상품설명'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=['category', 'name'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='보증_1020', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'category': 'deposit', 'name': 'm-정기적금', 'type': '상품설명'}, hash='54f0caaa6e60d0787a3b3c9742536eafa275a53f6ea6893244e301b694fead20'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='d41e70af-3b4c-40fb-aaa4-11c255d539b0', node_type=<ObjectType.TEXT: '1'>, metadata={'category': 'deposit', 'name': 'WELCOME 잔돈모아올림적금', 'type': '상품설명'}, hash='3f478246dfb5972774c65e4b3e38ea163b905f577bdd431cd8f97ab0ceedd53f'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='39e8d2ea-ae4e-480d-9663-b9c4add8abc5', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='1d1e5133b2a924366cff06532df2810bb994ce0d1a3596b739d3f3c1054601cd')},

In [107]:
object_dict = dict()
object_dict['a'] = 1
object_dict['b'] = 2
object_dict['c'] = 3

In [108]:
list(object_dict.values())

[1, 2, 3]